In [71]:
pip install "mesa>=3.3"


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [72]:
pip install ipykernel tqdm matplotlib seaborn pandas numpy


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [73]:
# Requiero Mesa > 3.3
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
from mesa import Agent, Model

# Debido a que necesitamos que existe un solo agente por celda, elegimos ''SingleGrid''.
from mesa.space import SingleGrid

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# Haremos uso de ''batch_run'' para ejecutar varias simulaciones (si quieres usarlo después).
from mesa.batchrunner import batch_run

# matplotlib lo usaremos para animaciones rápidas en Jupyter (opcional).
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()

import json
import random

In [74]:
class Stack:
    def __init__(self):
        self.__data = []

    def empty(self):
        return not self.__data

    def clear(self):
        self.__data.clear()

    def push(self, element):
        self.__data.append(element)

    def pop(self):
        if self.__data:  # not empty
            return self.__data.pop()
        else:
            raise Exception("No such element")

    def top(self):
        if self.__data:  # not empty
            return self.__data[-1]
        else:
            raise Exception("No such element")

    def display(self):
        print(self.__data)

In [75]:
class BomberoSimpson(Agent):
    """
    Agente que representa a un miembro de la familia Simpson.
    Puede usar dos estrategias:
    - 'random'
    - 'zonas'
    """

    def __init__(self, model, nombre, zona, estrategia="random"):
        super().__init__(model)
        self.nombre = nombre         # Homero, Marge, Bart, Lisa, Maggie, Abuelo
        self.zona = zona             # 0, 1 o 2 (la casa se particiona en 3 columnas)
        self.estrategia = estrategia # 'random' o 'zonas'

        # Estado interno
        self.rol_actual = "apagafuegos"   # o "rescatador"
        self.tiene_victima = False
        self.id_victima = None

        # Métricas opcionales
        self.celdas_visitadas = 1
        self.fuegos_apagados = 0
        self.victimas_encontradas = 0
        self.victimas_rescatadas = 0

    # ---------- utilidades internas ----------

    def _vecinos_disponibles(self):
        """
        Regresa una lista de celdas vecinas (Von Neumann) que estén libres dentro de la grilla.
        - No atraviesa paredes.
        - Si el bombero lleva una víctima, no entra en fuego.
        """
        posibles = self.model.grid.get_neighborhood(
            self.pos,
            moore=False,
            include_center=True  # incluir la celda actual por si decide quedarse
        )
        vecinos = []
        for (x, y) in posibles:
            if (0 <= x < self.model.grid.width) and (0 <= y < self.model.grid.height):

                # 1) Paredes: no se puede mover a una pared
                if self.model.celdas_paredes[x][y]:
                    continue

                # 2) Si está cargando víctima, no puede entrar a fuego
                nivel_destino = self.model.celdas_fuego[x][y]
                if self.tiene_victima and nivel_destino == 2:
                    # casilla en llamas → prohibida mientras cargue víctima
                    continue

                # 3) Solo una familia Simpson por celda
                if self.model.grid.is_cell_empty((x, y)) or (x, y) == self.pos:
                    vecinos.append((x, y))

        random.shuffle(vecinos)
        return vecinos

    # ----------- interacción por estrategia -----------

    def _rescatar_si_esta_en_base(self):
        """
        Reglas comunes: si el bombero está en la base y trae víctima, la rescata.
        Se usa tanto en random como en zonas.
        """
        if self.tiene_victima and self.id_victima is not None and self.pos == self.model.ambulancia_pos:
            victima = self.model.lista_victimas[self.id_victima]
            victima["estado"] = "rescatada"
            victima["turno_rescate"] = self.model.turno_actual
            self.victimas_rescatadas += 1
            self.tiene_victima = False
            self.id_victima = None

    def _interactuar_con_celda_random(self):
        """
        Estrategia RANDOM:
        - Si está en la base y lleva víctima → la rescata.
        - Si pisa fuego/humo:
            - Si lleva víctima -> la deja (si la celda está libre) y luego apaga.
            - Siempre apaga el fuego/humo.
        - Si hay víctima viva y no lleva otra -> la carga.
        - Si hay puerta, la abre.
        - NO intenta ir a la ambulancia de forma deliberada.
        """
        x, y = self.pos

        # 0) Rescatar si está en base con víctima
        self._rescatar_si_esta_en_base()

        # 1) Puertas: 0 = no hay, 1 = cerrada, 2 = abierta
        if self.model.celdas_puertas[x][y] == 1:
            self.model.celdas_puertas[x][y] = 2  # abrir la puerta

        # 2) Fuego / humo en la celda actual
        nivel_fuego = self.model.celdas_fuego[x][y]

        if nivel_fuego > 0:
            # Si lleva víctima: la deja en la celda SI está libre de otras víctimas
            if self.tiene_victima and self.id_victima is not None:
                if self.model.celdas_victimas[x][y] == -1:
                    v = self.model.lista_victimas[self.id_victima]
                    v["x"], v["y"] = x, y
                    v["estado"] = "viva"
                    self.model.celdas_victimas[x][y] = self.id_victima
                    self.tiene_victima = False
                    self.id_victima = None

            # Apaga el fuego/humo (en nuestra versión lo dejamos en vacío directo)
            self.model.celdas_fuego[x][y] = 0
            self.fuegos_apagados += 1

        # 3) Víctima en la celda (después de apagar, por si la dejó aquí)
        idx_victima = self.model.celdas_victimas[x][y]
        if idx_victima != -1:
            victima = self.model.lista_victimas[idx_victima]
            if victima["estado"] == "viva":
                self.victimas_encontradas += 1
                # Si no lleva otra víctima, la carga
                if not self.tiene_victima:
                    self.tiene_victima = True
                    self.id_victima = idx_victima
                    self.model.celdas_victimas[x][y] = -1
                    victima["estado"] = "cargada"
                    victima["por_quien"] = self.nombre

    def _interactuar_con_celda_zonas(self):
        """
        Interacción en estrategia por zonas:
        - Si está en la base y lleva víctima -> la rescata.
        - Apaga fuego/humo si está en la celda.
        - Si es rescatador y hay víctima viva y no lleva otra -> la carga.
        - Puertas se abren si se pisan.
        """
        x, y = self.pos

        # 0) Rescatar si está en base con víctima
        self._rescatar_si_esta_en_base()

        # 1) Abrir puerta si hay
        if self.model.celdas_puertas[x][y] == 1:
            self.model.celdas_puertas[x][y] = 2

        # 2) Fuego / humo
        nivel_fuego = self.model.celdas_fuego[x][y]
        if nivel_fuego > 0:
            self.model.celdas_fuego[x][y] = 0
            self.fuegos_apagados += 1

        # 3) Víctima en la celda
        idx_victima = self.model.celdas_victimas[x][y]
        if idx_victima != -1:
            victima = self.model.lista_victimas[idx_victima]
            if victima["estado"] == "viva":
                self.victimas_encontradas += 1

                if self.estrategia == "zonas" and self.rol_actual == "rescatador" and not self.tiene_victima:
                    # Levanta a la víctima
                    self.tiene_victima = True
                    self.id_victima = idx_victima
                    self.model.celdas_victimas[x][y] = -1
                    victima["estado"] = "cargada"
                    victima["por_quien"] = self.nombre

    def _interactuar_con_celda(self):
        """
        Llama a la variante correcta según la estrategia.
        """
        if self.estrategia == "random":
            self._interactuar_con_celda_random()
        else:
            self._interactuar_con_celda_zonas()

    # ---------- estrategias de movimiento ----------

    def _mover_random(self):
        """
        Movimiento totalmente aleatorio dentro de la grilla (sin atravesar paredes
        y respetando la regla de no entrar a fuego cargando víctima).
        Después del movimiento aplica la lógica de interacción RANDOM.
        """
        vecinos = self._vecinos_disponibles()
        if not vecinos:
            return

        nueva_pos = vecinos[0]
        if nueva_pos != self.pos:
            self.model.grid.move_agent(self, nueva_pos)
            self.celdas_visitadas += 1

        # Interactúa con lo que haya en la nueva celda
        self._interactuar_con_celda()

    def _celda_mas_cercana(self, objetivos):
        """
        Recibe una lista de coordenadas [(x,y), ...] y regresa el objetivo
        más cercano (Manhattan) a la posición actual.
        Si la lista está vacía, regresa None.
        """
        if not objetivos:
            return None

        x0, y0 = self.pos
        mejor_obj = None
        mejor_dist = None

        for (x, y) in objetivos:
            dist = abs(x - x0) + abs(y - y0)
            if (mejor_dist is None) or (dist < mejor_dist):
                mejor_dist = dist
                mejor_obj = (x, y)

        return mejor_obj

    def _mover_hacia_objetivo(self, objetivo):
        """
        Un paso greedy hacia un objetivo (x,y).
        Si no puede acercarse, intenta un movimiento random.
        Respeta paredes y la restricción de no entrar a fuego cargando víctima.
        """
        if objetivo is None:
            self._mover_random()
            return

        x0, y0 = self.pos
        xt, yt = objetivo

        candidatos = self._vecinos_disponibles()
        if not candidatos:
            return

        mejor = self.pos
        mejor_dist = abs(x0 - xt) + abs(y0 - yt)

        for (nx, ny) in candidatos:
            dist = abs(nx - xt) + abs(ny - yt)
            if dist < mejor_dist:
                mejor_dist = dist
                mejor = (nx, ny)

        if mejor != self.pos:
            self.model.grid.move_agent(self, mejor)
            self.celdas_visitadas += 1

        # Después de moverse hacia el objetivo (zonas),
        # interactúa con fuego / víctimas / puertas / base.
        self._interactuar_con_celda()

    def _step_zonas(self):
        """
        Estrategia por zonas:
        - El modelo calcula cuántos fuegos hay en cada zona.
        - Si en mi zona hay fuego -> rol 'apagafuegos' y me acerco al fuego más cercano.
        - Si no hay fuego -> rol 'rescatador':
            - si llevo víctima, voy hacia la ambulancia;
            - si no llevo, voy hacia la víctima viva más cercana.
        """
        # Rol dinámico depende de la cantidad de fuego en mi zona
        total_fuego_zona = self.model.fuego_por_zona[self.zona]
        if total_fuego_zona > 0:
            self.rol_actual = "apagafuegos"
        else:
            self.rol_actual = "rescatador"

        if self.rol_actual == "apagafuegos":
            lista_fuegos = self.model.obtener_fuegos_en_zona(self.zona)
            objetivo = self._celda_mas_cercana(lista_fuegos)
            self._mover_hacia_objetivo(objetivo)

        else:  # rescatador
            if self.tiene_victima:
                # Llevarla a la ambulancia
                objetivo = self.model.ambulancia_pos
                self._mover_hacia_objetivo(objetivo)
                # La función de interacción se encarga de rescatar si llega a la base
            else:
                # Buscar víctima viva más cercana en mi zona
                lista_vivas = self.model.obtener_victimas_vivas_en_zona(self.zona)
                objetivo = self._celda_mas_cercana(lista_vivas)
                self._mover_hacia_objetivo(objetivo)

    # ---------- paso del agente ----------

    def step(self):
        """
        Llamado por el modelo en cada turno.
        """
        if self.estrategia == "random":
            self._mover_random()
        elif self.estrategia == "zonas":
            self._step_zonas()
        else:
            self._mover_random()

In [76]:
class SimpsonsFlashpointModel(Model):
    """
    Modelo simplificado de Flashpoint con temática de Los Simpsons.
    """

    def __init__(self,
                 width=8,
                 height=7,
                 estrategia="random",
                 num_victimas=6,
                 prob_fuego_inicial=0.1,
                 prob_humo_inicial=0.1):
        super().__init__()

        self.width = width
        self.height = height
        self.estrategia = estrategia  # 'random' o 'zonas'

        # Grilla con una sola familia Simpson por celda
        self.grid = SingleGrid(width, height, torus=False)

        # Matrices de fuego/humo y víctimas
        self.celdas_fuego = np.zeros((width, height), dtype=int)
        self.celdas_victimas = -1 * np.ones((width, height), dtype=int)

        # -------- NUEVO: paredes y puertas --------
        # Paredes: True = pared (no se puede entrar)
        self.celdas_paredes = np.zeros((width, height), dtype=bool)

        # Puertas: 0 = no hay, 1 = cerrada, 2 = abierta
        self.celdas_puertas = np.zeros((width, height), dtype=int)

        # Aquí más adelante podrás "pintar" paredes y puertas, por ejemplo:
        # self.celdas_paredes[3, 2] = True
        # self.celdas_puertas[4, 1] = 1  # puerta cerrada inicialmente
        # ------------------------------------------

        # Lista con info de víctimas (personajes de la serie)
        nombres_victimas = [
            "Milhouse", "Nelson", "Flanders", "Krusty", "Skinner", "Moe",
            "Barney", "Ralph", "Carl", "Lenny"
        ]
        random.shuffle(nombres_victimas)
        self.lista_victimas = []

        # Ambulancia: por simplicidad, una esquina del mapa
        self.ambulancia_pos = (0, 0)

        # Turno actual
        self.turno_actual = 0

        # Conteo de fuego por zona
        self.fuego_por_zona = [0, 0, 0]

        # Crear fuego/humo inicial de forma aleatoria
        for x in range(width):
            for y in range(height):
                r = random.random()
                if r < prob_fuego_inicial:
                    self.celdas_fuego[x][y] = 2  # fuego
                elif r < prob_fuego_inicial + prob_humo_inicial:
                    self.celdas_fuego[x][y] = 1  # humo

        # Colocar víctimas en celdas aleatorias sin repetir
        total_celdas = width * height
        indices_celdas = list(range(total_celdas))
        random.shuffle(indices_celdas)

        for i in range(num_victimas):
            if i >= len(indices_celdas):
                break
            idx_celda = indices_celdas[i]
            x = idx_celda % width
            y = idx_celda // width

            # Evitar la celda de la ambulancia
            if (x, y) == self.ambulancia_pos:
                continue

            self.celdas_victimas[x][y] = len(self.lista_victimas)
            self.lista_victimas.append({
                "id": f"victima_{i}",
                "nombre": nombres_victimas[i],
                "x": x,
                "y": y,
                "estado": "viva",          # viva / cargada / rescatada
                "por_quien": None,
                "turno_rescate": None
            })

        # Familia Simpson: 6 bomberos
        self.nombres_simpsons = [
            "Homero", "Marge", "Bart", "Lisa", "Maggie", "Abuelo"
        ]

        # Zona de cada uno (2 bomberos por zona)
        zonas_asignadas = [0, 0, 1, 1, 2, 2]

        self.agents  # AgentSet que viene del Model de Mesa (ya lo crea Mesa)

        for i, nombre in enumerate(self.nombres_simpsons):
            zona = zonas_asignadas[i]
            bombero = BomberoSimpson(
                model=self,
                nombre=nombre,
                zona=zona,
                estrategia=estrategia
            )

            # Asignar posición inicial en la zona correspondiente
            colocado = False
            intentos = 0
            while not colocado and intentos < 100:
                if estrategia == "zonas":
                    # División vertical en 3 zonas
                    ancho_zona = width // 3
                    x_min = zona * ancho_zona
                    x_max = (zona + 1) * ancho_zona - 1
                    if zona == 2:
                        x_max = width - 1  # por si no divide exacto
                else:
                    # En random pueden aparecer en cualquier lado
                    x_min, x_max = 0, width - 1

                x = self.random.randrange(x_min, x_max + 1)
                y = self.random.randrange(0, height)

                # No los coloques en una pared
                if self.celdas_paredes[x][y]:
                    intentos += 1
                    continue

                if self.grid.is_cell_empty((x, y)):
                    self.grid.place_agent(bombero, (x, y))
                    self.agents.add(bombero)
                    colocado = True
                intentos += 1

        # DataCollector opcional (para análisis)
        self.datacollector = DataCollector(
            model_reporters={
                "FuegoTotal": lambda m: int((m.celdas_fuego == 2).sum()),
                "VictimasVivas": lambda m: sum(1 for v in m.lista_victimas if v["estado"] == "viva"),
                "VictimasRescatadas": lambda m: sum(1 for v in m.lista_victimas if v["estado"] == "rescatada")
            },
            agent_reporters={
                "FuegosApagados": lambda a: a.fuegos_apagados,
                "VictimasRescatadas": lambda a: a.victimas_rescatadas
            }
        )

    # ----- utilidades por zona -----

    def _actualizar_fuego_por_zona(self):
        """
        Cuenta cuántas celdas con fuego hay en cada zona.
        """
        self.fuego_por_zona = [0, 0, 0]
        ancho_zona = self.width // 3

        for x in range(self.width):
            for y in range(self.height):
                if self.celdas_fuego[x][y] == 2:
                    if x < ancho_zona:
                        self.fuego_por_zona[0] += 1
                    elif x < 2 * ancho_zona:
                        self.fuego_por_zona[1] += 1
                    else:
                        self.fuego_por_zona[2] += 1

    def obtener_fuegos_en_zona(self, zona):
        """
        Regresa la lista de coordenadas (x,y) con fuego en la zona indicada.
        """
        celdas = []
        ancho_zona = self.width // 3

        for x in range(self.width):
            for y in range(self.height):
                if self.celdas_fuego[x][y] == 2:
                    if zona == 0 and x < ancho_zona:
                        celdas.append((x, y))
                    elif zona == 1 and ancho_zona <= x < 2 * ancho_zona:
                        celdas.append((x, y))
                    elif zona == 2 and x >= 2 * ancho_zona:
                        celdas.append((x, y))
        return celdas

    def obtener_victimas_vivas_en_zona(self, zona):
        """
        Regresa la lista de coordenadas de víctimas vivas en una zona.
        """
        celdas = []
        ancho_zona = self.width // 3
        for v in self.lista_victimas:
            if v["estado"] == "viva":
                x, y = v["x"], v["y"]
                if zona == 0 and x < ancho_zona:
                    celdas.append((x, y))
                elif zona == 1 and ancho_zona <= x < 2 * ancho_zona:
                    celdas.append((x, y))
                elif zona == 2 and x >= 2 * ancho_zona:
                    celdas.append((x, y))
        return celdas

    # ----- paso global del modelo -----

    def step(self):
        """
        Un turno del juego:
        - Se recalcula fuego por zona (para la estrategia de zonas).
        - Se colectan datos.
        - Se actualizan los agentes.
        """
        self._actualizar_fuego_por_zona()
        self.datacollector.collect(self)
        self.agents.shuffle_do("step")
        self.turno_actual += 1

{
  "turno": 0,
  "estrategia": "random",
  "bomberos": [
    { "id": "Homero", "x": 3, "y": 4, "zona": 0, "rol": "apagafuegos", "tiene_victima": false },
    ...
  ],
  "fuego": [ { "x": 1, "y": 2, "nivel": 2 }, ... ],
  "humo":  [ { "x": 3, "y": 1 }, ... ],
  "victimas": [
    { "id": "victima_0", "nombre": "Milhouse", "x": 5, "y": 2, "estado": "viva" },
    { "id": "victima_1", "nombre": "Flanders", "x": -1, "y": -1, "estado": "rescatada" }
  ],
  "ambulancia": { "x": 0, "y": 0 }
}

In [77]:
def construir_snapshot(modelo):
    """
    Crea un diccionario con el estado del modelo en el turno actual.
    """
    snap = {
        "turno": modelo.turno_actual,
        "estrategia": modelo.estrategia,
        "bomberos": [],
        "fuego": [],
        "humo": [],
        "victimas": [],
        "ambulancia": {
            "x": int(modelo.ambulancia_pos[0]),
            "y": int(modelo.ambulancia_pos[1])
        }
    }

    # Bomberos (Simpsons)
    for agente in modelo.agents:
        if isinstance(agente, BomberoSimpson):
            x, y = agente.pos
            snap["bomberos"].append({
                "id": agente.nombre,
                "x": int(x),
                "y": int(y),
                "zona": int(agente.zona),
                "rol": agente.rol_actual,
                "tiene_victima": bool(agente.tiene_victima)
            })

    # Fuego y humo
    width, height = modelo.width, modelo.height
    for x in range(width):
        for y in range(height):
            nivel = int(modelo.celdas_fuego[x][y])
            if nivel == 2:
                snap["fuego"].append({"x": int(x), "y": int(y), "nivel": 2})
            elif nivel == 1:
                snap["humo"].append({"x": int(x), "y": int(y)})

    # Víctimas
    for v in modelo.lista_victimas:
        # Si está "cargada" no tiene posición propia (va con el agente)
        estado = v["estado"]
        if estado in ["viva", "rescatada"]:
            snap["victimas"].append({
                "id": v["id"],
                "nombre": v["nombre"],
                "x": int(v["x"]) if estado == "viva" else -1,
                "y": int(v["y"]) if estado == "viva" else -1,
                "estado": estado
            })
        elif estado == "cargada":
            snap["victimas"].append({
                "id": v["id"],
                "nombre": v["nombre"],
                "x": -1,
                "y": -1,
                "estado": estado
            })

    return snap


def simular_y_exportar_json(estrategia, pasos, nombre_archivo):
    """
    Crea un modelo con la estrategia indicada, corre 'pasos' turnos
    y escribe un JSON con todos los snapshots.
    """
    modelo = SimpsonsFlashpointModel(estrategia=estrategia)
    lista_turnos = []

    for _ in range(pasos):
        snap = construir_snapshot(modelo)
        lista_turnos.append(snap)
        modelo.step()

    # Agregar último estado después del último step (opcional)
    snap_final = construir_snapshot(modelo)
    lista_turnos.append(snap_final)

    datos = {"turnos": lista_turnos}

    with open(nombre_archivo, "w") as f:
        json.dump(datos, f, indent=2)

    print(f"Archivo JSON guardado en: {nombre_archivo}")

In [78]:
simular_y_exportar_json(
    estrategia="random",
    pasos=150,
    nombre_archivo="simpsons_flashpoint_random.json"
)

simular_y_exportar_json(
    estrategia="zonas",
    pasos=150,
    nombre_archivo="simpsons_flashpoint_zonas.json"
)

Archivo JSON guardado en: simpsons_flashpoint_random.json
Archivo JSON guardado en: simpsons_flashpoint_zonas.json
